<a href="https://colab.research.google.com/github/JAlonMor/Webscraping/blob/main/Git_WebScrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Instalo fake_useragent

! pip install fake_useragent
! pip install xmltojson
! pip install utils

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=e90f9b780655e4fca2653d259ddd4b34683ac606c5084b42799e6a1a1ac21ff5
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [3]:
import requests
import json
import csv
import pandas as pd
import numpy as np
from IPython.core.display import HTML
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from urllib import request as urllib2
import re
from fake_useragent import UserAgent
import random

import time

import http.client  # or http.client if you're on Python 3
http.client._MAXHEADERS = 1000

In [4]:
# Funciones

def check_url(pagina):
    
    languaje = ['fr-CH, fr', 'en-US,es', 'es-ES,es', 'en-US,en', 'de-CH', 'es']
    languaje_header = random.choice(languaje)
    
    url_base = 'https://www.coches.net/segunda-mano/?MakeId=28&ModelId=8&MinYear=2014&pg=' +str(pagina)
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    ua=UserAgent()
    hdr = {'User-Agent': ua.random,
          'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
          'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
          'Accept-Encoding': 'none',
          'Accept-Language': languaje_header, #+ ';q=0.9,en;q=0.8'
          'Connection': 'keep-alive'}

    r = requests.get(url_base, headers=hdr)
    html = r.content
    status = r.status_code
    return status, html


In [7]:
# Defino archivo de salida
f_csv = 'coches_net.csv'#'coches_net.csv'


with open(f_csv, 'w') as f:

  writer = csv.writer(f)

  writer.writerow(['titulo', 'precio', 'kilometros', 'año', 'combustible', 'ciudad', 'garantia', 'url'])

  # Contador segun status html
  status = 0
  pagina = 0
  es_ultima = False

  # Genero una búsqueda en las primeras 65 páginas, con lo obtendré más de 2000 resultados
  
  while es_ultima == False:

    pagina = pagina +1
    
    time.sleep(random.randrange(7,22,1))
    url_result = check_url(pagina)
    html_url = url_result[1]
    status = len(html_url)

    if status >= 10000:
      
      print('pagina ', pagina)
        
    
      # Convertir a BeatifulSoup
      soup  = BeautifulSoup(html_url,'lxml')

      # Compruebo si es la última página
      paginacion = soup.find_all('li', attrs={'class':'sui-MoleculePagination-item'})
      paginacion
      pagina_siguiente = paginacion[-1].find('a').get('href')

      if 'null' in pagina_siguiente:
        es_ultima == True
      else:
        es_ultima == False


      scripts = soup.find_all('script')
      script_coches = scripts[-2]

      # Extraigo script
      js = 'window = {};\n'+script_coches.text.strip()+';\nprocess.stdout.write(JSON.stringify(window.__INITIAL_PROPS__));'

      # Exporto a archivo temporal .js y creo objeto JSON
      with open('temp.js','w') as f:
        f.write(js)

      from subprocess import check_output
      script_JS = check_output(['node','temp.js'])

      info_coches_js = json.loads(script_JS)

      # cálculo número de anuncios en cada pagina
      items = len(info_coches_js['initialResults']['items'])
    
      # Itero por todos los elementos de interés del JSON
      for item in range(0,items-1):

        titulo = info_coches_js['initialResults']['items'][item]['title'] if 'title' in info_coches_js['initialResults']['items'][item] else None
        precio = info_coches_js['initialResults']['items'][item]['price'] if 'price' in info_coches_js['initialResults']['items'][item] else None
        kilometros = info_coches_js['initialResults']['items'][item]['km'] if 'km' in info_coches_js['initialResults']['items'][item] else None
        año = info_coches_js['initialResults']['items'][item]['year'] if 'year' in info_coches_js['initialResults']['items'][item] else None
        combustible = info_coches_js['initialResults']['items'][item]['fuelTypeId'] if 'fuelTypeId' in info_coches_js['initialResults']['items'][item] else None
        ciudad = info_coches_js['initialResults']['items'][item]['location']['mainProvince'] if 'mainProvince' in info_coches_js['initialResults']['items'][item]['location'] else None
        garantia = info_coches_js['initialResults']['items'][item]['hasWarranty'] if 'hasWarranty' in info_coches_js['initialResults']['items'][item] else None
        url = info_coches_js['initialResults']['items'][item]['url'] if 'url' in info_coches_js['initialResults']['items'][item] else None

        writer.writerow([titulo, precio, kilometros, año, combustible, ciudad, garantia, url])

    else:

      time.sleep(30 + random.randrange(17,200,1))
      pagina = pagina - 1
      es_ultima == False
      
    
    
print(status)
print('FIN')











pagina  1


KeyboardInterrupt: ignored